In [1]:
import torch
import sys
from tqdm import tqdm
from model import get_stage

In [2]:
whole_model_dict = torch.load("../whole_llama2_model.pt")
whole_model_keys_lst = list(whole_model_dict.keys())

In [3]:
for k,v in whole_model_dict.items():
    print(k, v.shape)

tok_embeddings.weight torch.Size([32000, 8192])
norm.weight torch.Size([8192])
output.weight torch.Size([32000, 8192])
layers.0.attention.wq.weight torch.Size([8192, 8192])
layers.0.attention.wk.weight torch.Size([1024, 8192])
layers.0.attention.wv.weight torch.Size([1024, 8192])
layers.0.attention.wo.weight torch.Size([8192, 8192])
layers.0.feed_forward.w1.weight torch.Size([28672, 8192])
layers.0.feed_forward.w2.weight torch.Size([8192, 28672])
layers.0.feed_forward.w3.weight torch.Size([28672, 8192])
layers.0.attention_norm.weight torch.Size([8192])
layers.0.ffn_norm.weight torch.Size([8192])
layers.1.attention.wq.weight torch.Size([8192, 8192])
layers.1.attention.wk.weight torch.Size([1024, 8192])
layers.1.attention.wv.weight torch.Size([1024, 8192])
layers.1.attention.wo.weight torch.Size([8192, 8192])
layers.1.feed_forward.w1.weight torch.Size([28672, 8192])
layers.1.feed_forward.w2.weight torch.Size([8192, 28672])
layers.1.feed_forward.w3.weight torch.Size([28672, 8192])
layers.

In [4]:
whole_model_dict["rope.freqs"]

tensor([1.0000e+00, 8.6719e-01, 7.5000e-01, 6.4844e-01, 5.6250e-01, 4.8633e-01,
        4.2188e-01, 3.6523e-01, 3.1641e-01, 2.7344e-01, 2.3730e-01, 2.0508e-01,
        1.7773e-01, 1.5430e-01, 1.3379e-01, 1.1523e-01, 1.0010e-01, 8.6426e-02,
        7.5195e-02, 6.4941e-02, 5.6152e-02, 4.8584e-02, 4.2236e-02, 3.6621e-02,
        3.1738e-02, 2.7344e-02, 2.3682e-02, 2.0508e-02, 1.7822e-02, 1.5381e-02,
        1.3306e-02, 1.1536e-02, 1.0010e-02, 8.6670e-03, 7.5073e-03, 6.5002e-03,
        5.6152e-03, 4.8828e-03, 4.2114e-03, 3.6469e-03, 3.1586e-03, 2.7313e-03,
        2.3651e-03, 2.0599e-03, 1.7776e-03, 1.5411e-03, 1.3351e-03, 1.1520e-03,
        9.9945e-04, 8.6594e-04, 7.5150e-04, 6.4850e-04, 5.6076e-04, 4.8637e-04,
        4.2152e-04, 3.6430e-04, 3.1662e-04, 2.7466e-04, 2.3746e-04, 2.0504e-04,
        1.7738e-04, 1.5354e-04, 1.3351e-04, 1.1539e-04], dtype=torch.bfloat16)

In [6]:
model_lst = []
for i in tqdm(range(8)):
    model_lst.append(getattr(get_stage, "model_"+str(i))().bfloat16())
model_lst

100%|██████████| 8/8 [00:13<00:00,  1.68s/it]


[model_0(
   (tok_embeddings): VocabEmbedding()
   (layers): ModuleList(
     (0-9): 10 x TransformerBlock(
       (attention): Attention(
         (wq): noParallelLinear()
         (wk): noParallelLinear()
         (wv): noParallelLinear()
         (wo): noParallelLinear()
       )
       (feed_forward): FeedForward(
         (w1): noParallelLinear()
         (w2): noParallelLinear()
         (w3): noParallelLinear()
       )
       (attention_norm): RMSNorm()
       (ffn_norm): RMSNorm()
     )
   )
 ),
 model_1(
   (layers): ModuleList(
     (0-9): 10 x TransformerBlock(
       (attention): Attention(
         (wq): noParallelLinear()
         (wk): noParallelLinear()
         (wv): noParallelLinear()
         (wo): noParallelLinear()
       )
       (feed_forward): FeedForward(
         (w1): noParallelLinear()
         (w2): noParallelLinear()
         (w3): noParallelLinear()
       )
       (attention_norm): RMSNorm()
       (ffn_norm): RMSNorm()
     )
   )
 ),
 model_2(
   (la

In [7]:
for i in range(8):
    print(i)
    state_dict = model_lst[i].state_dict()
    new_state_dict = {}
    for k, v in state_dict.items():
        if "layers" in k:
            new_k = k.lstrip("layers.")
            layer_id = int(new_k.split(".")[0])
            tmp_k = new_k.lstrip(str(layer_id))
            layer_id += 10*(i)
            new_k = "layers." + str(layer_id) + tmp_k
            new_state_dict[k] = whole_model_dict[new_k]
        else:
            new_state_dict[k] = whole_model_dict[k]
    model_lst[i].load_state_dict(new_state_dict)            

0
1
2
3
4
5
6
7


In [11]:
model_lst[0].state_dict()['tok_embeddings.weight']

tensor([[ 2.9564e-04,  2.8491e-05,  7.5340e-05,  ..., -2.8729e-05,
         -2.9755e-04,  4.8161e-05],
        [-1.2207e-04, -7.2861e-04,  1.0834e-03,  ..., -3.7956e-04,
          3.8452e-03,  1.9989e-03],
        [ 3.3379e-04,  2.6855e-03, -1.8463e-03,  ..., -1.4496e-03,
         -6.1646e-03,  1.1169e-02],
        ...,
        [ 5.4321e-03,  1.8555e-02,  9.0332e-03,  ..., -1.4343e-02,
          3.4180e-03, -1.3550e-02],
        [-2.7924e-03, -7.9346e-03,  1.4801e-03,  ...,  1.7456e-02,
         -1.6235e-02,  1.6724e-02],
        [ 1.0498e-02, -6.8283e-04,  2.7771e-03,  ...,  9.9487e-03,
         -4.7302e-03,  4.1504e-03]], dtype=torch.bfloat16)

In [14]:
for i, model_ in enumerate(tqdm(model_lst)):
    torch.save(model_.state_dict(), "pp_model/stage_"+str(i)+".pt")

100%|██████████| 8/8 [05:26<00:00, 40.84s/it]
